# Common imports

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import h5py
from PIL import Image

In [2]:
IR_file_1 = '/home/hangzheng/tissue_segmentation/data/Jintea_TMA/Array3_ir.mat'
IR_file_2 = '/home/hangzheng/tissue_segmentation/data/Jintea_TMA/Array4_ir.mat'
CL_file_1 = '/home/hangzheng/tissue_segmentation/data/Jintea_TMA/Array3_cl.mat'
CL_file_2 = '/home/hangzheng/tissue_segmentation/data/Jintea_TMA/Array4_cl.mat'


In [34]:

IR1 = h5py.File(IR_file_1,'r')['X']
# IR2 = h5py.File(IR_file_2,'r')['X']





In [3]:
CL1 = h5py.File(CL_file_1,'r')['CL']
CL1 = np.array(CL1)
IR1 = h5py.File(IR_file_1,'r')['X']
IR1 = np.array(IR1)

In [4]:
plt.imshow(CL2/7*255)
plt.show()

# Provided functions

In [4]:
def line_plot(pt1, pt2):
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')
    
def get_input(im,i,j,area):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """

    plt.figure()
    plt.imshow(im[i*area[0]:(i+1)*area[0],j*area[1]:(j+1)*area[1]])
    plt.show()
    square_shape = 500
    dist = square_shape//2
    pts = []
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(1, timeout=0, show_clicks=True)
        if not clicked:
            break
        # Unpack user inputs and save as homogeneous coordinates
        pt = np.array([clicked[0][0], clicked[0][1]])
        
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        
        left_up = np.array([pt[0]-dist,pt[1]-dist])
        left_down = np.array([pt[0]+dist,pt[1]-dist])
        right_up = np.array([pt[0]-dist,pt[1]+dist])
        right_down = np.array([pt[0]+dist,pt[1]+dist])
        
        line_plot(left_up,left_down)
        line_plot(left_down,right_down)
        line_plot(right_down, right_up)
        line_plot(right_up,left_up)
        pt = [pt[1]+area[0]*i,pt[0]+area[1]*j]
        print(pt)
        pts.append(pt)
    return pts

# Your implementation

# Main function

In [6]:
im = CL2

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
pts = []

area = [5000,4000]

for i in range(0, im.shape[0]//area[0]+1):
    for j in range(0, im.shape[1]//area[1]+1):
        print(f'Getting area {i} {j}')
        pt = get_input(im, i,j,area)
        pts.extend(pt)



Getting area 0 0
Click the two endpoints, use the right key to undo, and use the middle key to stop input
[612.1771644013024, 1213.6828348724912]
Click the two endpoints, use the right key to undo, and use the middle key to stop input
[690.8496568668979, 1849.1145047869195]
Click the two endpoints, use the right key to undo, and use the middle key to stop input
[551.65986250469, 3906.7027692717356]
Click the two endpoints, use the right key to undo, and use the middle key to stop input
[1259.712294695053, 1081.5243188777677]
Click the two endpoints, use the right key to undo, and use the middle key to stop input
[1332.3330569709879, 1783.52502087847]
Click the two endpoints, use the right key to undo, and use the middle key to stop input
[1217.3501833674245, 2509.732643637817]
Click the two endpoints, use the right key to undo, and use the middle key to stop input
[1308.1261362123432, 3187.526424879874]
Click the two endpoints, use the right key to undo, and use the middle key to stop 

In [7]:
with open('center2.txt','w') as f:
    for idx, line in enumerate(pts):
        x = str(line[0])
        y = str(line[1])
        f.writelines(f"{idx} {x} {y}\n")

In [8]:
CL_store_path = '/home/hangzheng/tissue_segmentation/data/Jintea/CL'
IR_store_path = '/home/hangzheng/tissue_segmentation/data/Jintea/IR'
# IR2 = h5py.File(IR_file_2,'r')['X']
# IR2 = np.array(IR2)
dis = 260
import os

neg = np.where(IR1<0)
IR1[neg] = 0
with open('center.txt', 'r') as f:
    for i,line in enumerate(f.readlines()):
        _, x, y = line.split()
        x = int(float(x))
        y = int(float(y))
        print(i,x,y)
        im_array = CL1[x-dis:x+dis,y-dis:y+dis]
        im = im_array/7*255
        im = np.uint8(im)
        im = Image.fromarray(im)
        im.save(os.path.join(CL_store_path, 'image', f'{i+1}.png'))   
        np.save(os.path.join(CL_store_path,f'{i+1}'), im_array)
        
        im_array = IR1[:,x-dis:x+dis,y-dis:y+dis]
        im = im_array[4]/np.max(im_array[4])*255
        im = np.uint8(im)
        im = Image.fromarray(im)
        im.save(os.path.join(IR_store_path, 'image', f'{i+1}.png')) 
        np.save(os.path.join(IR_store_path,f'{i+1}'), im_array)

0 817 572
1 799 1292
2 938 2018
3 811 2738
4 963 3446
5 1489 535
6 1495 1274
7 1634 1970
8 1513 2696
9 2179 523
10 2155 1219
11 2276 3337
12 2839 445
13 2869 1183
14 2966 1897
15 2869 2611
16 2972 3307
17 3510 2551
18 3644 3241
19 4164 348
20 4158 1098
21 4279 1788
22 4176 2520
23 672 4953
24 709 5667
25 854 6381
26 817 7059
27 896 7773
28 1374 5653
29 1532 6355
30 1574 7789
31 2028 4898
32 2052 5631
33 2203 6314
34 2155 7041
35 2264 7761
36 2693 4850
37 2766 5570
38 2887 6321
39 2851 7004
40 2948 7725
41 3347 4808
42 3517 6962
43 3625 7682
44 4013 4783
45 4037 5528
46 4194 6200
47 4188 6908
48 4285 7640
49 823 9107
50 932 10475
51 1513 9101
52 1683 9797
53 1592 11159
54 2197 9047
55 2191 9785
56 2234 10499
57 2240 11123
58 2863 9053
59 2887 9803
60 2893 10457
61 2948 11177
62 3498 9035
63 3535 9749
64 3607 10433
65 3656 11141
66 4200 8986
67 4231 9779
68 5612 414
69 5575 1122
70 5715 1830
71 5642 2581
72 5775 3283
73 6296 426
74 6259 1092
75 6417 1794
76 6320 2514
77 6417 3228
78 7119

In [3]:
CL_store_path = '/home/hangzheng/tissue_segmentation/data/Jintea/CL'
IR_store_path = '/home/hangzheng/tissue_segmentation/data/Jintea/IR'
CL2 = h5py.File(CL_file_2,'r')['CL']
CL3 = np.array(CL2)
IR2 = h5py.File(IR_file_2,'r')['X']
IR2 = np.array(IR2)
dis = 260
import os

neg = np.where(IR2<0)
IR2[neg] = 0
with open('center2.txt', 'r') as f:
    for i,line in enumerate(f.readlines()):
        _, x, y = line.split()
        x = int(float(x))
        y = int(float(y))
        print(i,x,y)
        im_array = CL2[x-dis:x+dis,y-dis:y+dis]
        im = im_array/7*255
        im = np.uint8(im)
        im = Image.fromarray(im)
        im.save(os.path.join(CL_store_path, 'image', f'{i+1+294}.png'))   
        np.save(os.path.join(CL_store_path,f'{i+1+294}'), im_array)
        
        im_array = IR2[:,x-dis:x+dis,y-dis:y+dis]
        im = im_array[4]/np.max(im_array[4])*255
        im = np.uint8(im)
        im = Image.fromarray(im)
        im.save(os.path.join(IR_store_path, 'image', f'{i+1+294}.png')) 
        np.save(os.path.join(IR_store_path,f'{i+1+294}'), im_array)

0 612 1213
1 690 1849
2 551 3906
3 1259 1081
4 1332 1783
5 1217 2509
6 1308 3187
7 1187 3865
8 1931 1057
9 1870 2479
10 2010 3139
11 1870 3822
12 2578 1014
13 2639 1686
14 2603 3090
15 3208 984
16 3341 1656
17 3190 2406
18 3293 3084
19 3135 3780
20 3922 1601
21 4878 972
22 4733 1694
23 4811 3102
24 315 5449
25 412 6775
26 267 7519
27 981 5322
28 957 7489
29 1653 5340
30 1592 6103
31 1713 6920
32 1628 7495
33 2373 6805
34 2300 7501
35 2929 5310
36 2929 6060
37 2990 6756
38 3595 5340
39 3607 6109
40 3668 6817
41 4812 5110
42 4865 5884
43 4792 6585
44 4889 7352
45 436 8209
46 1096 8172
47 1786 8195
48 2433 8177
49 3087 8207
50 4890 8032
51 491 9499
52 464 10158
53 491 10831
54 634 11485
55 1111 9479
56 1105 10132
57 1124 10858
58 1262 11524
59 1778 9472
60 1784 10152
61 1758 10877
62 1902 11570
63 2431 9512
64 2431 10139
65 2529 11609
66 3058 9518
67 3052 10145
68 3032 10890
69 3143 11629
70 3718 9583
71 3738 10237
72 3725 10962
73 3829 11642
74 4979 9466
75 4886 10145
76 4986 10903
77 55

In [4]:
from glob import glob
import shutil
np.random.seed(42)
split_list = [i for i in range(1,len(glob('/home/hangzheng/tissue_segmentation/data/Jintea/IR/*npy'))+1)]
np.random.shuffle(split_list)
split = int((len(split_list)*0.97))
train_list = split_list[:split]
test_list = split_list[split:]

train_path = '/home/hangzheng/tissue_segmentation/data/Jintea/train'
val_path = '/home/hangzheng/tissue_segmentation/data/Jintea/val_large'
for i, idx in enumerate(train_list):
    IR = f'/home/hangzheng/tissue_segmentation/data/Jintea/IR/{idx}.npy'
    CL = f'/home/hangzheng/tissue_segmentation/data/Jintea/CL/{idx}.npy'
    shutil.copy(IR,os.path.join(train_path,'IR',f'{i+1}.npy'))
    shutil.copy(CL,os.path.join(train_path,'CL',f'{i+1}.npy'))

for i, idx in enumerate(test_list):
    IR = f'/home/hangzheng/tissue_segmentation/data/Jintea/IR/{idx}.npy'
    CL = f'/home/hangzheng/tissue_segmentation/data/Jintea/CL/{idx}.npy'
    shutil.copy(IR,os.path.join(val_path,'IR',f'{i+1}.npy'))
    shutil.copy(CL,os.path.join(val_path,'CL',f'{i+1}.npy'))


In [5]:
from glob import glob
import numpy as np
import os
CL_paths = sorted(glob('/home/hangzheng/tissue_segmentation/data/Jintea/val_large/CL/*npy'))
IR_paths = sorted(glob('/home/hangzheng/tissue_segmentation/data/Jintea/val_large/IR/*npy'))
CL_save = '/home/hangzheng/tissue_segmentation/data/Jintea/val/CL'
CL_image_save = '/home/hangzheng/tissue_segmentation/data/Jintea/val/CL/image'
IR_save = '/home/hangzheng/tissue_segmentation/data/Jintea/val/IR'
IR_image_save = '/home/hangzheng/tissue_segmentation/data/Jintea/val/IR/image'
for n, path in enumerate(CL_paths):
    CL = np.load(path)
    center= CL.shape[0]//2
    image_shape = 224
    for i in range(2):
        for j in range(2):
            patch = CL[center+(i-1)*image_shape:center+i*image_shape,center+(j-1)*image_shape:center+j*image_shape]
            np.save(f'{CL_save}/{n*4+i*2+j+1}',patch)
            im = patch/7*255
            im = np.uint8(im)
            im = Image.fromarray(im)
            im.save(os.path.join(CL_image_save, f'{n*4+i*2+j+1}.png')) 

for n, path in enumerate(IR_paths):
    IR = np.load(path)
    center= IR.shape[1]//2
    image_shape = 224
    for i in range(2):
        for j in range(2):
            patch = IR[:,center+(i-1)*image_shape:center+i*image_shape,center+(j-1)*image_shape:center+j*image_shape]
            np.save(f'{IR_save}/{n*4+i*2+j+1}',patch)
            im = patch[6]/np.max(patch[6])*255
            im = np.uint8(im)
            im = Image.fromarray(im)
            im.save(os.path.join(IR_image_save, f'{n*4+i*2+j+1}.png')) 
        
    